## Importing necessary libraries

In [1]:
%matplotlib inline
import random
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

## Helper Functions

In [2]:
def colour(n):
    ret = {}
    for i in range(n):
        r = int(random.random() * 256)
        g = int(random.random() * 256)
        b = int(random.random() * 256)
        ret[i] = "#{:02x}{:02x}{:02x}".format(r,g,b) 
    return ret
colorList = colour(5000)
minDegNode = lambda G: min(G, key=G.degree)
maxDegNode = lambda G: max(G, key=G.degree)

def transformDict(myDict):
    for key,value in myDict.items():
        myDict[key] = colorList[value]
    return myDict

## List of Strategies Used:
1. Greedy Graph Coloring Algorithm (Vanilla)
2. Greedy Graph Coloring By Ordering Nodes By Largest Degree First
3. Greedy Graph Coloring By Random Ordering of Nodes
4. Welsh Powell
5. Greedy Graph Colouring Using BFS
6. Greedy Graph Colouring Using DFS

In [3]:
def greedyGraphColoring(G):
    colors = {}
    graphNodes = G.nodes()
    
    for node in graphNodes:
        adjColors = set()
        for adjNode in G.neighbors(node):
            if adjNode in colors:
                adjColors.add(colors[adjNode])
                
        for color in itertools.count():
            if color not in adjColors:
                break
                
        colors[node] = color
    return colors

def greedyDegreeSort(G):
    colors = {}
    graphNodes = list(G.nodes())
    graphNodes.sort(key=lambda node: -G.degree(node))
    
    for node in graphNodes:
        adjColors = set()
        for adjNode in G.neighbors(node):
            if adjNode in colors:
                adjColors.add(colors[adjNode])
                
        for color in itertools.count():
            if color not in adjColors:
                break
                
        colors[node] = color
    return colors

def greedyRandomShuffling(G):
    colors = {}
    graphNodes = G.nodes()
    random.shuffle(list(graphNodes))
    
    for node in graphNodes:
        adjColors = set()
        for adjNode in G.neighbors(node):
            if adjNode in colors:
                adjColors.add(colors[adjNode])
                
        for color in itertools.count():
            if color not in adjColors:
                break
                
        colors[node] = color
    return colors

if(False):
        def welsh_powell(G):  
            x = sorted(G.degree, key=lambda x: x[1], reverse=True)
            len_g = len(G)
            no_colored = 0
            k = 1
            colors = dict()
            while no_colored <  len_g:
                colored = set()
                colorednodes = set()
                for node in x:
                    y = set(G.neighbors(node[0]))
                    if y.intersection(colorednodes) == set():
                        colors[node[0]] = k
                        no_colored +=1
                        colored.add(node)
                        colorednodes.add(node[0])
                x = list(set(x) - colored)
                k+=1
            return colors
    
def welsh_powell(G):
    colors = {}  
    x = sorted(G.degree, key=lambda x: x[1], reverse=True)
    len_g = len(G)
    no_colored = 0
    k = 1
    while no_colored <  len_g:
        colored = set()
        for new_node in x:
            if colored not in G.neighbors(new_node[0]):
                colors[new_node[0]] = k
                no_colored +=1
                colored.add(new_node)
        x = list(set(x) - colored)
        k+=1
    return colors

def strategy_connected_sequential_bfs(G, colors):
    return strategy_connected_sequential(G, colors, 'bfs')
def strategy_connected_sequential_dfs(G, colors):
    return strategy_connected_sequential(G, colors, 'dfs')
def strategy_connected_sequential(G, colors, traversal='bfs'):
    for component_graph in nx.connected_component_subgraphs(G):
        source = component_graph.nodes()[0]
        yield source
        if traversal == 'bfs':
            tree = nx.bfs_edges(component_graph, source)
        elif traversal == 'dfs':
            tree = nx.dfs_edges(component_graph, source)
        for (_, end) in tree:
            yield end
            
def greedy_color(G, strategy, interchange=False):
    colors = {}
    for node in nodes:
        neighbour_colors = set()
        for neighbour in G.neighbors_iter(node):
            if neighbour in colors:
                neighbour_colors.add(colors[neighbour])
        for color in itertools.count():
            if color not in neighbour_colors:
                break
        colors[node] = color
    return colors
            


## Function to check time taken by various strategies:

In [4]:
def printGraphAndAnalyse(G, sizeoffigure):
    fig = plt.figure(figsize=(2*sizeoffigure,sizeoffigure))
    plt.subplot(1, 2, 1)
    nx.draw(G, with_labels=True)
    plt.subplot(1, 2, 2)
    nodeColors = transformDict(greedyGraphColoring(G))
    nx.draw(G, with_labels=True, node_color=list(nodeColors.values()))
    plt.show()
    print("Time taken by different Algorithms:\n")
    print("Basic Greedy Algorithm:")
    %timeit -n 10 -r 2 greedyGraphColoring(G)
    print("\nGreedy Graph Coloring By Ordering Nodes (By Largest Degree First):")
    %timeit -n 10 -r 2 greedyDegreeSort(G)
    print("\nGreedy Algorithm With Random Shuffling:")
    %timeit -n 10 -r 2 greedyRandomShuffling(G)
    print("\nWelsh Powell:")
    %timeit -n 10 -r 2 welsh_powell(G)
    print("\nGreedy Algorithm using DFS ")
    %timeit -n 10 -r 2 nx.coloring.greedy_color(G, strategy=nx.coloring.strategy_connected_sequential_dfs)    
    print("\nGreedy Algorithm using BFS ")
    %timeit -n 10 -r 2 nx.coloring.greedy_color(G, strategy=nx.coloring.strategy_connected_sequential_bfs)

In [5]:
def graph_colouring(Type, noOfNodes, plotSize, edgeProbability, branchingFactor_bt, height_bt):
    if Type == "Complete Graph" :
        G = nx.complete_graph(noOfNodes)
    elif Type == "Balanced Tree":
        G = nx.balanced_tree(branchingFactor_bt, height_bt)
    elif Type == "Cycle Graph":
        G = nx.cycle_graph(noOfNodes)
    elif Type == "Random graph":
        G = nx.fast_gnp_random_graph(noOfNodes,edgeProbability)
    printGraphAndAnalyse(G, plotSize)
    
style = {'description_align':'left','description_width': '25%'}
layout = widgets.Layout(width='90%')
layout1 = widgets.Layout(width='85%')
cuf = True
interactive_plot = widgets.interactive(graph_colouring,
    noOfNodes = widgets.IntSlider(description='Number of nodes in graph', min = 2, max = 100, step = 1, value = 3, style = style, layout = layout, continuous_update = cuf),
    Type = widgets.Dropdown(options=['Complete Graph','Balanced Tree', 'Cycle Graph', 'Random graph'], value='Complete Graph', description='Type of Graph:', style = style, layout = layout1),
    plotSize = widgets.IntSlider(description='Size of Graph (Inch x Inch)', min = 3, max = 15, step = 1, value = 10, style = style, layout = layout, continuous_update = cuf),
    edgeProbability = widgets.FloatSlider(description='Edge Probabibility (Erdős–Rényi model)', min = 0, max = 6.6, step = 0.01, value = 0.4, style = style, layout = layout, continuous_update = cuf),
    branchingFactor_bt = widgets.IntSlider(description='Branching Factor of the Balanced Tree)', min = 1, max = 10, step = 1, value = 3, style = style, layout = layout, continuous_update = cuf),
    height_bt = widgets.IntSlider(description='Height of the Balanced Tree)', min = 1, max = 10, step = 1, value = 2, style = style, layout = layout, continuous_update = cuf))
output = interactive_plot.children[-1]
output.layout.height = '1100px'
interactive_plot

interactive(children=(Dropdown(description='Type of Graph:', layout=Layout(width='85%'), options=('Complete Gr…